<a href="https://colab.research.google.com/github/anshavvayeda/MLOps_Assignment/blob/main/Ml_Ops_Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
imimportport torch.nn as nn
import torch.optim as optim
import torch.quantization
import time
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [ ]:
digits = load_digits()

X = digits.data
y = digits.target
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)


In [ ]:
class PyTorchLogisticRegression(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(PyTorchLogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.linear(x)

input_dim = X_train.shape[1]
output_dim = len(np.unique(y_train))  # Number of classes
pytorch_model = PyTorchLogisticRegression(input_dim, output_dim)


In [ ]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(pytorch_model.parameters(), lr=0.01)

# Train the model
num_epochs = 100
for epoch in range(num_epochs):
    optimizer.zero_grad()
    outputs = pytorch_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()


In [ ]:
# Measure inference time for the original model
start_inference = time.time()
with torch.no_grad():
    test_outputs = pytorch_model(X_test_tensor)
    _, predicted = torch.max(test_outputs, 1)
    inference_time = time.time() - start_inference

# Calculate accuracy
original_accuracy = (predicted == y_test_tensor).sum().item() / len(y_test_tensor)

# Calculate model size
original_model_size = len(pickle.dumps(pytorch_model))

# Display original model results
print(f"Original Model Accuracy: {original_accuracy:.4f}")
print(f"Original Model Size: {original_model_size} bytes")
print(f"Original Model Inference Time: {inference_time:.6f} seconds")


Original Model Accuracy: 0.7556
Original Model Size: 4048 bytes
Original Model Inference Time: 0.022371 seconds


In [ ]:
# Apply dynamic quantization
quantized_pytorch_model = torch.quantization.quantize_dynamic(
    pytorch_model, {nn.Linear}, dtype=torch.qint8
)

# Measure quantized model inference time
start_inference_quantized = time.time()
with torch.no_grad():
    quantized_outputs = quantized_pytorch_model(X_test_tensor)
    _, quantized_predicted = torch.max(quantized_outputs, 1)
    quantized_inference_time = time.time() - start_inference_quantized

# Calculate quantized model accuracy
quantized_accuracy = (quantized_predicted == y_test_tensor).sum().item() / len(y_test_tensor)

# Calculate the quantized model size
quantized_model_size = len(pickle.dumps(quantized_pytorch_model))

# Display quantized model results
print(f"Quantized Model Accuracy: {quantized_accuracy:.4f}")
print(f"Quantized Model Size: {quantized_model_size} bytes")
print(f"Quantized Model Inference Time: {quantized_inference_time:.6f} seconds")


Quantized Model Accuracy: 0.7528
Quantized Model Size: 2537 bytes
Quantized Model Inference Time: 0.045655 seconds


In [ ]:
# Model Comparison
print(f"\nModel Comparison:")
print(f"Metric                   Original Model      Quantized Model")
print(f"------------------------------------------------------------")
print(f"Accuracy                 {original_accuracy:.4f}              {quantized_accuracy:.4f}")
print(f"Model Size (bytes)       {original_model_size}                {quantized_model_size}")
print(f"Inference Time (s)       {inference_time:.6f}          {quantized_inference_time:.6f}")



Model Comparison:
Metric                   Original Model      Quantized Model
------------------------------------------------------------
Accuracy                 0.7556              0.7528
Model Size (bytes)       4048                2537
Inference Time (s)       0.022371          0.045655
